# Imports and Constants

In [1]:
%tensorflow_version 2.x

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf

import csv
import random
import string

In [3]:
LOW_NOTE = -3
HIGH_NOTE = 8
CONTINUE_NOTE = 0 - LOW_NOTE
NUM_NOTE_TYPES = HIGH_NOTE - LOW_NOTE + 1

NUM_PREV_NOTES = 8

CSV_COLUMN_NAMES = ["Note"]

for i in range(0, NUM_PREV_NOTES):
  CSV_COLUMN_NAMES.append("Back-" + str(i + 1))

CSV_COLUMN_NAMES.extend(["Chord", "Chord-Next", "Beat"])

BEATS_PER_MEASURE = 8
MEASURE_DATA_LENGTH = BEATS_PER_MEASURE + 1
HEADER_DATA_LENGTH = 3
MEASURES_PER_MELODY = 4
BEATS_PER_MELODY = MEASURES_PER_MELODY * BEATS_PER_MEASURE
DATA_LENGTH = HEADER_DATA_LENGTH + MEASURES_PER_MELODY * MEASURE_DATA_LENGTH

TRAINED_KEY_TYPE = "Minor"

# Training Data

In [4]:
def note_to_int(note, octave_shift):
  i = 0

  if note.isdigit() or (len(note) >= 2 and note[0] == "-" and note[1:].isdigit()):
    if int(note) + octave_shift * 7 < 0:
      i = int(note) + octave_shift * 7

    else:
      i = int(note) + 1 + octave_shift * 7
    
  return i - LOW_NOTE

In [13]:
data = csv.writer(open("music_training_data.csv", "w"))
data.writerow(CSV_COLUMN_NAMES)

melodies = csv.reader(open("music_melody_chart.csv", "r"))

data_row = [None] * len(CSV_COLUMN_NAMES)

starting_data_list = []

for row in melodies:
  for shift in range(-1, 2):
    if row[2] == TRAINED_KEY_TYPE:
      chords = []
      chord = MEASURES_PER_MELODY - 1

      for i in range(0, MEASURES_PER_MELODY):
        chords.append(note_to_int(row[HEADER_DATA_LENGTH + MEASURE_DATA_LENGTH * i], 0))

      for i in range(0, NUM_PREV_NOTES + 1):
        data_row[i] = note_to_int(row[-1 - i], shift)

      data_row[NUM_PREV_NOTES + 1] = chords[-1]
      data_row[NUM_PREV_NOTES + 2] = chords[0]
      data_row[NUM_PREV_NOTES + 3] = BEATS_PER_MEASURE - 1

      starting_data_list.append([chords, data_row[0:NUM_PREV_NOTES]])

      for n in range(0, BEATS_PER_MELODY):
        data_row[NUM_PREV_NOTES + 3] = int((data_row[NUM_PREV_NOTES + 3] + 1) % BEATS_PER_MEASURE)

        if n % BEATS_PER_MEASURE == 0:
          chord = (chord + 1) % MEASURES_PER_MELODY

          data_row[NUM_PREV_NOTES + 2] = chords[(chord + 1) % 4]
          data_row[NUM_PREV_NOTES + 1] = chords[chord]

        for r in range(NUM_PREV_NOTES, 0, -1):
          data_row[r] = data_row[r-1]

        data_row[0] = note_to_int(row[n + chord + 1 + HEADER_DATA_LENGTH], shift)
        
        valid_entry = True

        for r in range(0, NUM_PREV_NOTES + 3):
          if data_row[r] < 0 or data_row[r] >= NUM_NOTE_TYPES:
            valid_entry = False
            break
        
        if valid_entry:
          data.writerow(data_row)

train = pd.read_csv("music_training_data.csv", names=CSV_COLUMN_NAMES, header=0)
train_y = train.pop("Note")

print(train)

      Back-1  Back-2  Back-3  Back-4  ...  Back-8  Chord  Chord-Next  Beat
0          3       2       3       3  ...       5      4           7     0
1          4       3       2       3  ...       3      4           7     1
2          3       4       3       2  ...       3      4           7     2
3          3       3       4       3  ...       4      4           7     3
4          6       3       3       4  ...       3      4           7     4
...      ...     ...     ...     ...  ...     ...    ...         ...   ...
4741       3       3       3       3  ...       3      9           7     3
4742       3       3       3       3  ...       3      9           7     4
4743       3       3       3       3  ...       3      9           7     5
4744      11       3       3       3  ...       3      9           7     6
4745       3       3       3       3  ...       7      4           7     0

[4746 rows x 11 columns]


In [14]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [15]:
my_feature_columns = []

for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [16]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[40, 40],
    n_classes=NUM_NOTE_TYPES)

classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpt8bzymjp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer ins

# Song Generation

In [17]:
def input_fn_pred(features, batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

In [18]:
def int_list_to_chord_progression(int_list, key, key_type):
  s = ""

  for i in int_list:
    s += int_to_chord(i, key, key_type) + " "

  return s

In [19]:
def int_list_to_melody(int_list, key, key_type):
  s = ""

  for i in range(0, 32):
    if i % BEATS_PER_MEASURE == 0:
      s += "| "

    s += int_to_note_text(int_list[i], key, key_type) + " "

  s += "|"

  return s

In [20]:
def int_to_note_letter_text(dist_from_c, does_use_sharps):
  if dist_from_c % 12 == 1:
    return "C#" if does_use_sharps else "Db"
  
  elif dist_from_c % 12 == 2:
    return "D"

  elif dist_from_c % 12 == 3:
    return "D#" if does_use_sharps else "Eb"

  elif dist_from_c % 12 == 4:
    return "E"

  elif dist_from_c % 12 == 5:
    return "F"

  elif dist_from_c % 12 == 6:
    return "F#" if does_use_sharps else "Gb"

  elif dist_from_c % 12 == 7:
    return "G"

  elif dist_from_c % 12 == 8:
    return "G#" if does_use_sharps else "Ab"

  elif dist_from_c % 12 == 9:
    return "A"

  elif dist_from_c % 12 == 10:
    return "A#" if does_use_sharps else "Bb"

  elif dist_from_c % 12 == 11:
    return "B"

  return "C"

In [21]:
def int_to_note_text(i, key, key_type):
  note = ""

  p = int_to_pitch(i, key, key_type)
  s = does_key_use_sharps(key, key_type)

  if p == -1:
    return "-"

  note += int_to_note_letter_text(p, s) + str(int(p / 12))

  return note

In [22]:
def does_key_use_sharps(key, key_type):
  k = key % 12

  if key_type == "Major":
    return k == 0 or k == 2 or k == 4 or k == 6 or k == 7 or k == 9 or k == 11

  else:
    return k == 1 or k == 4 or k == 6 or k == 8 or k == 9 or k == 11

In [23]:
def int_to_pitch(i, key, key_type):
  i += LOW_NOTE

  if i == 0 or (key_type != "Major" and key_type != "Minor"):
    return -1

  elif i > 0:
    i -= 1

  pitch = key + 36

  for p in range(min(0, i), max(0, i)):
    if (key_type == "Major" and (p % 7 == 2 or p % 7 == 6)) or (key_type == "Minor" and (p % 7 == 1 or p % 7 == 4)):
      pitch += (1 if i > 0 else -1)
    
    else:
      pitch += (2 if i > 0 else -2)
  
  return pitch

In [24]:
def int_to_chord(i, key, key_type):
  return int_to_note_text(i, key, key_type)[:-1]

In [25]:
CORRECTION_CONSTANT = 0.4
CONTINUE_CORRECTION_CONSTANT = 0.5
CAN_START_ON_CONTINUE = True
NUM_CONSIDERED_NOTES = 4

print("Which key would you like?")

for k in range(0, 12):
  print(str(k + 1) + ": " + int_to_chord(1 - LOW_NOTE, k, TRAINED_KEY_TYPE) + " " + TRAINED_KEY_TYPE)

print("13: Choose for me!")

key_input = 0
chosen_key = random.choice([0, 2, 5, 7, 9])

while key_input < 1 or key_input > 13:
  ipt = input()

  if ipt.isnumeric():
    key_input = int(ipt)

if key_input < 13:
  chosen_key = key_input - 1

print("\nKey:\n" + int_to_chord(1 - LOW_NOTE, chosen_key, TRAINED_KEY_TYPE) + " " + TRAINED_KEY_TYPE)

print("\n\nWhich chord progression would you like?")

chord_progression_options = []

for d in range(0, len(starting_data_list)):
  new_chord = True

  for chord_progression in chord_progression_options:
    if chord_progression[0] == starting_data_list[d][0]:
      chord_progression[1].append(d)

      new_chord = False
      break
  
  if new_chord:
    chord_progression_options.append([starting_data_list[d][0], [d]])

    print(str(len(chord_progression_options)) + ": " + int_list_to_chord_progression(starting_data_list[d][0], chosen_key, TRAINED_KEY_TYPE))

print(str(len(chord_progression_options) + 1) + ": Choose for me!")

progression_input = 0

starting_data = random.choice(starting_data_list)

while progression_input < 1 or progression_input > len(chord_progression_options) + 1:
  ipt = input()

  if ipt.isnumeric():
    progression_input = int(ipt)

if progression_input < len(chord_progression_options) + 1:
  starting_data = starting_data_list[random.choice(chord_progression_options[progression_input - 1][1])]

print("\nChord Progression:\n" + int_list_to_chord_progression(starting_data[0], chosen_key, TRAINED_KEY_TYPE) + "\n\n")

data_row = [0]
data_row.extend(starting_data[1])
data_row.extend(starting_data[0][0:2])
data_row.append(0)

if data_row[NUM_PREV_NOTES] == CONTINUE_NOTE:
  data_row[NUM_PREV_NOTES] = starting_data[0][-1]

melody = []

streak_count = 0
streak_note = -1

for n in range(0, BEATS_PER_MELODY):
  predict = {}
  count = 0

  for feature in CSV_COLUMN_NAMES:
    if feature != "Note":
      predict[feature] = [data_row[count]]

    count += 1

  predictions = classifier.predict(input_fn=lambda: input_fn_pred(predict))

  for pred_dict in predictions:
    used_notes = []
    possible_notes = []
    possible_notes_probabilities = []

    pred_dict["probabilities"][CONTINUE_NOTE] *= CONTINUE_CORRECTION_CONSTANT

    if streak_note >= 0:
      pred_dict["probabilities"][streak_note] *= CORRECTION_CONSTANT ** streak_count

    for i in range(0, NUM_NOTE_TYPES):
      max_prob = 0
      max_note = -1

      for m in range(0, len(pred_dict["probabilities"])):
        if pred_dict["probabilities"][m] > max_prob and m not in used_notes:
          max_prob = pred_dict["probabilities"][m]
          max_note = m
      
      used_notes.append(max_note)

      if n != 0 or max_note != CONTINUE_NOTE or CAN_START_ON_CONTINUE:
        possible_notes.append(max_note)
        possible_notes_probabilities.append(max_prob)

        if len(possible_notes) == NUM_CONSIDERED_NOTES + 1:
          break

  total_probability = 0
  
  for i in range(0, NUM_CONSIDERED_NOTES + 1):
    possible_notes_probabilities[i] -= possible_notes_probabilities[NUM_CONSIDERED_NOTES]
    total_probability += possible_notes_probabilities[i]

  rand = random.random() * total_probability

  probability_cumulative = 0
  note = 0

  for i in range(0, NUM_CONSIDERED_NOTES + 1):
    probability_cumulative += possible_notes_probabilities[i]

    if rand <= probability_cumulative or i == NUM_CONSIDERED_NOTES:
      note = possible_notes[i]
      break

  if note == streak_note:
    streak_count += 1

  else:
    streak_note = note
    streak_count = 1

  if data_row[NUM_PREV_NOTES + 3] == BEATS_PER_MEASURE - 1 and int((n + 1) / BEATS_PER_MEASURE) < MEASURES_PER_MELODY:
    data_row[NUM_PREV_NOTES + 2] = starting_data[0][(int((n + 1) / BEATS_PER_MEASURE) + 1) % 4]
    data_row[NUM_PREV_NOTES + 1] = starting_data[0][int((n + 1) / BEATS_PER_MEASURE)]

  for r in range(NUM_PREV_NOTES, 0, -1):
    data_row[r] = data_row[r-1]

  data_row[0] = note
  data_row[NUM_PREV_NOTES + 3] = int((data_row[NUM_PREV_NOTES + 3] + 1) % 8)

  melody.append(note)

print("\n\nKey:\n" + int_to_chord(1 - LOW_NOTE, chosen_key, TRAINED_KEY_TYPE) + " " + TRAINED_KEY_TYPE + "\n")
print("Chord Progression:\n" + int_list_to_chord_progression(starting_data[0], chosen_key, TRAINED_KEY_TYPE) + "\n")
print("Melody:\n" + int_list_to_melody(melody, chosen_key, TRAINED_KEY_TYPE))

Which key would you like?
1: C Minor
2: C# Minor
3: D Minor
4: Eb Minor
5: E Minor
6: F Minor
7: F# Minor
8: G Minor
9: G# Minor
10: A Minor
11: Bb Minor
12: B Minor
13: Choose for me!
6

Key:
F Minor


Which chord progression would you like?
1: F Bb Db Eb 
2: F Ab Eb Bb 
3: F Db Ab Eb 
4: F Eb Ab Db 
5: F Ab Db Eb 
6: F Eb Ab Ab 
7: F Ab Eb Db 
8: F Bb Eb Ab 
9: F Bb Eb C 
10: F Ab C Eb 
11: F Eb Db Eb 
12: F Ab Bb Db 
13: F Db Ab G 
14: F Ab C Bb 
15: F Db Eb Ab 
16: F Db Eb C 
17: F F Eb Bb 
18: F Eb Db Bb 
19: F Bb Eb F 
20: F Bb Eb Eb 
21: F Db Eb Bb 
22: F Ab C Db 
23: F Db Bb C 
24: F Bb Eb Bb 
25: Choose for me!
15

Chord Progression:
F Db Eb Ab 


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpt8bzymjp/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling mode